In [1]:
import pandas as pd

In [12]:
df = pd.read_csv('houses_msk.csv')

In [13]:
df = df.drop(columns=['key', 'bedrooms_number', 'offer_type', 'living_area', 'kitchen_area', 'floor_number', 'passenger_lifts_number', 'cargo_lifts_number', 'balconies_number', 'loggias_number', 'underground'])
df.dropna(subset = ['land', 'year', 'material_type'], inplace=True)
df['total_floors'] = df['total_floors'].fillna(1.0)

In [14]:
df

,region,repair,price,total_area,land,total_floors,year,material_type,lng,lat
0,Moscow,design,44000000,596.0,2300.0,3.0,2020.0,brick,37.337199,55.438864
1,Moscow,design,62475000,641.0,3100.0,3.0,2020.0,brick,37.336507,55.437526
10,Moscow,design,90900000,370.0,1400.0,3.0,2015.0,monolith,37.550064,55.922449
16,Moscow,design,35000000,440.0,1200.0,3.0,2018.0,brick,37.264606,55.405976
22,Moscow,design,30000000,200.0,700.0,3.0,2021.0,brick,37.464365,55.530429
...,...,...,...,...,...,...,...,...,...,...
2360,Moscow,design,42000000,355.0,604.0,3.0,2019.0,brick,37.415389,55.656900
2361,Moscow,design,42000000,441.0,1500.0,1.0,2007.0,brick,37.463395,55.534143
2362,Moscow,design,43000000,516.0,3800.0,1.0,2010.0,brick,37.321415,55.401852
2363,Moscow,design,43000000,533.0,1200.0,3.0,2005.0,brick,37.469090,55.626955


In [15]:
# Changing lat and lng to dist and azmth

def get_azimuth(lat, lng):
 
    rad = 6372795

    llat1 = city_center_coordinates[0]
    llong1 = city_center_coordinates[1]
    llat2 = lat
    llong2 = lng

    lat1 = llat1*math.pi/180.
    lat2 = llat2*math.pi/180.
    long1 = llong1*math.pi/180.
    long2 = llong2*math.pi/180.

    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)

    x = (cl1*sl2) - (sl1*cl2*cdelta)
    y = sdelta*cl2
    z = math.degrees(math.atan(-y/x))

    if (x < 0):
        z = z+180.

    z2 = (z+180.) % 360. - 180.
    z2 = - math.radians(z2)
    anglerad2 = z2 - ((2*math.pi)*math.floor((z2/(2*math.pi))) )
    angledeg = (anglerad2*180.)/math.pi
    
    return round(angledeg, 2)

In [16]:
from geopy.distance import geodesic
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder

city_center_coordinates = [55.7522, 37.6156]
df['distance'] = list(map(lambda x, y: geodesic(city_center_coordinates, [x, y]).meters, df['lat'], df['lng']))
df['azimuth'] = list(map(lambda x, y: get_azimuth(x, y), df['lat'], df['lng']))     

df = df.loc[(df['distance'] < 60000)] 
df['distance'] = df['distance'].round(0)
df['azimuth'] = df['azimuth'].round(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 988 entries, 0 to 2365
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   region         988 non-null    object 
 1   repair         988 non-null    object 
 2   price          988 non-null    int64  
 3   total_area     988 non-null    float64
 4   land           988 non-null    float64
 5   total_floors   988 non-null    float64
 6   year           988 non-null    float64
 7   material_type  988 non-null    object 
 8   lng            988 non-null    float64
 9   lat            988 non-null    float64
 10  distance       988 non-null    float64
 11  azimuth        988 non-null    float64
dtypes: float64(8), int64(1), object(3)
memory usage: 88.8+ KB


<ipython-input-16-792fe6897fae>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance'] = df['distance'].round(0)
<ipython-input-16-792fe6897fae>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['azimuth'] = df['azimuth'].round(0)


In [17]:
# Proccessing statistical outliers
first_quartile = df.quantile(q=0.25)
third_quartile = df.quantile(q=0.75)
IQR = third_quartile - first_quartile
outliers = df[(df > (third_quartile + 1.5 * IQR)) | (df < (first_quartile - 1.5 * IQR))].count(axis=1)
outliers.sort_values(axis=0, ascending=False, inplace=True)

# Deleting some lines
outliers = outliers.head(70)
df.drop(outliers.index, inplace=True)

c:\python38-32\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [19]:
# Recategorizing objects and bools to int/float

categorical_columns = df.columns[df.dtypes == 'object']
labelencoder = LabelEncoder()
for column in categorical_columns:
    df[column] = labelencoder.fit_transform(df[column])
    print(dict(enumerate(labelencoder.classes_)))
    
bool_columns = df.columns[df.dtypes == 'bool']
labelencoder = LabelEncoder()
for column in bool_columns:
    df[column] = labelencoder.fit_transform(df[column])
    print(dict(enumerate(labelencoder.classes_)))
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918 entries, 0 to 2365
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   region         918 non-null    int32  
 1   repair         918 non-null    int32  
 2   price          918 non-null    int64  
 3   total_area     918 non-null    float64
 4   land           918 non-null    float64
 5   total_floors   918 non-null    float64
 6   year           918 non-null    float64
 7   material_type  918 non-null    int32  
 8   lng            918 non-null    float64
 9   lat            918 non-null    float64
 10  distance       918 non-null    float64
 11  azimuth        918 non-null    float64
dtypes: float64(8), int32(3), int64(1)
memory usage: 82.5 KB


In [22]:
df.to_csv('h_msk.csv', index=False)